In [2]:
#from wflow import wflow_bmi

import os
import time
import datetime
import pandas as pd
import xarray as xr

import hydrostats.data as hd
import hydrostats.visual as hv
import matplotlib.pyplot as plt

import HydroErr as he
import numpy as np

In [ ]:
#Set working directory
os.chdir("D:\sbranchett\jerom\wflow_rhine_sbm") 

In [ ]:
#Call BMI and initialize configuration file
wflow = wflow_bmi.wflowbmi_csdms()
wflow.initialize("wflow_sbm.ini")

In [ ]:
wflow.get_input_var_names()

In [ ]:
wflow.get_attribute_names()

In [ ]:
#List of output variables
wflow.get_output_var_names()

In [ ]:
#Set output variable name(s)
variable = "SurfaceRunoff" 

In [ ]:
#Get lon/lat for xarray
lon = wflow.get_grid_x(variable) 
lat = wflow.get_grid_y(variable)

In [ ]:
#Create 1D Lon/Lat lists for xarray
lon2 = lon[0,:]
lat2 = lat[:,0]

In [ ]:
#Get time step, start, end and calculate the amount of model time steps
tstep = wflow.get_time_step()
tstart = wflow.get_start_time()
tend = wflow.get_end_time()

tstep_nmbr = ((tend - tstart)/tstep) + 1

print tstep_nmbr
print time.ctime(int(tstart)) + " start"
print time.ctime(int(tend)) + " end"

In [ ]:
#Set correct date format that would work for all models
date_start = datetime.datetime.strptime(time.ctime(int(tstart)), "%a %b %d %H:%M:%S %Y").strftime('%Y-%m-%d')

In [ ]:
#Create xarray dataset with coordinates and time series
ds = xr.Dataset(coords={'lon': lon2,
                        'lat': lat2,
                        'time': pd.date_range(date_start,  periods = tstep_nmbr)})
ds

In [ ]:
#Create empty array for faster filling
data_variable = np.zeros(shape=(tstep_nmbr, len(lat2), len(lon2)))

In [ ]:
#Model run with retrieval of variables for each time step
for idx, i in enumerate(range(tstep_nmbr)):
    
    wflow.update_until(tstart + i *tstep)
    value = wflow.get_value(variable)
    data_variable[idx,:,:] = value

In [ ]:
#Append variable data to xarray dataset
ds[variable] = (('time', 'lat', 'lon'), data_variable)
ds

In [ ]:
#Plot variable map of time step 50
array = ds[variable].isel(time= 50)
array.plot()

In [ ]:
#Get timeseries data of a variabel from xarray Dataset for a given cell
timeseries = ds[variable].sel(lat= 51.8325, lon=6.0955, method= 'nearest')

In [ ]:
#Timeseries plot of selected variable
timeseries.plot()

In [ ]:
#Create dataframe with time, simulated and observed values
#Note: order of columns needs to be time, sim, obs for the hydrostats package
timeseries = ds[variable].sel(lat= 51.8325, lon=6.0955, method= 'nearest')

sim = timeseries.to_dataframe()

obs = pd.read_csv(r"C:\Users\jerom\Desktop\wflow\examples\grdc.csv", sep=';')
obs['date'] = pd.to_datetime(obs['date'])
obs['date'] = obs['date'].dt.strftime('%Y-%m-%d')

pd.to_datetime(obs['date'])
obs.set_index('date', inplace=True)

val = sim.merge(obs, left_index=True, right_index=True, how='inner')
val.columns = ['sim', 'lat', 'lon', 'obs']
val = val.drop(columns=['lat', 'lon'])

In [ ]:
#Plot hydrograph for simulated and observed values and calculate statistics

hv.plot(val,
      title='Hydrograph of Lobith',
      linestyles=['r-', 'k-'],
      legend=('Simulated', 'Observed'),
      labels=['Datetime', 'Streamflow (cms)'],
      metrics=['ME', 'NSE', 'SA'],
      grid=True)
plt.show()

In [ ]:
#Seperate statistics using hydrostats
sim = val['sim']
obs = val['obs']
  
nse = he.nse(sim, obs)
pearson_r = he.pearson_r(sim, obs)

print nse
print pearson_r